# TUGAS PRAKTIKUM

#### Tugas 1

Terdapat dataset mushroom. Berdasarkan dataset yang tersebut, bandingkan peforma antara algoritma Decision Tree dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [8]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [9]:
df = pd.read_csv('mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [10]:
df.isnull().sum()


class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [11]:
#split data
X = df.drop('class', axis=1)  # Menggunakan semua kolom kecuali 'class' sebagai fitur
y = df['class']  # Kolom 'class' sebagai target
y = y.map({'p': 1, 'e': 0})  # Encode label


# Melakukan One-Hot Encoding pada fitur-fitur kategorikal
X_encoded = pd.get_dummies(X, drop_first=True)
# Cek jumlah fitur dan instance
X.shape

(8124, 22)

In [21]:
from sklearn.preprocessing import OneHotEncoder

# Buat objek OneHotEncoder
encoder = OneHotEncoder()

# Fit dan transformasikan data kategorikal dalam kolom X
X_encoded = encoder.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


# Membuat objek DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Menentukan daftar parameter yang akan diuji
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Membuat objek GridSearchCV
grid_search_dt = GridSearchCV(estimator=dt_classifier, param_grid=param_grid_dt, cv=5, n_jobs=-1, verbose=1)

# Melatih model Decision Tree dengan data latih
grid_search_dt.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30, 40],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             verbose=1)

In [22]:
# Mendapatkan parameter terbaik dari hasil GridSearchCV
best_dt_params = grid_search_dt.best_params_

# Membuat ulang model Decision Tree dengan parameter terbaik
best_dt_model = DecisionTreeClassifier(random_state=42, **best_dt_params)

# Melatih ulang model Decision Tree dengan data latih
best_dt_model.fit(X_train, y_train)

# Memprediksi data uji dengan model Decision Tree terbaik
y_pred_dt = best_dt_model.predict(X_test)

# Mengukur akurasi
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Akurasi Decision Tree:", accuracy_dt)

# Menampilkan parameter terbaik
print("Parameter Terbaik Decision Tree:")
print(best_dt_params)

Akurasi Decision Tree: 1.0
Parameter Terbaik Decision Tree:
{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [23]:
# Inisialisasi model RandomForest
rf_model = RandomForestClassifier(random_state=42)

# Definisikan rentang hyperparameter yang akan diuji
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Gunakan GridSearchCV untuk mencari hyperparameter terbaik
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)



# Gunakan model RandomForest dengan hyperparameter terbaik
best_rf_model = grid_search_rf.best_estimator_

# Evaluasi akurasi RandomForest pada data pengujian
rf_accuracy = accuracy_score(y_test, best_rf_model.predict(X_test))
print("Akurasi RandomForest:", rf_accuracy)

# Cetak hyperparameter terbaik untuk RandomForest
best_rf_params = grid_search_rf.best_params_
print("Hyperparameter terbaik untuk RandomForest:", best_rf_params)

Akurasi RandomForest: 1.0
Hyperparameter terbaik untuk RandomForest: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


#### Tugas 2

Terdapat dataset mushroom. Berdasarkan dataset tersebut, bandingkan peforma antara algoritma Decision Tree dan AdaBoost. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [25]:
from sklearn.model_selection import train_test_split

# Preprocessing: Ubah label menjadi numerik
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

- Training Decision Tree

In [26]:
# Inisialisasi model Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)

# Definisikan rentang hyperparameter yang akan diuji
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Gunakan GridSearchCV untuk mencari hyperparameter terbaik
grid_search_dt = GridSearchCV(dt_model, param_grid_dt, cv=5, n_jobs=-1)
grid_search_dt.fit(X_train, y_train)

# Gunakan model Decision Tree dengan hyperparameter terbaik
best_dt_model = grid_search_dt.best_estimator_

# Evaluasi akurasi Decision Tree pada data pengujian
dt_accuracy = accuracy_score(y_test, best_dt_model.predict(X_test))
print("Akurasi Decision Tree:", dt_accuracy)

# Cetak hyperparameter terbaik untuk Decision Tree
best_dt_params = grid_search_dt.best_params_
print("Hyperparameter terbaik untuk Decision Tree:", best_dt_params)
     

Akurasi Decision Tree: 1.0
Hyperparameter terbaik untuk Decision Tree: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


- Training Model AdaBoost

In [55]:
# Inisialisasi model AdaBoost
adaboost_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), random_state=42)

# Definisikan rentang hyperparameter yang akan diuji
param_grid_adaboost = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0]
}

# Gunakan GridSearchCV untuk mencari hyperparameter terbaik
grid_search_adaboost = GridSearchCV(adaboost_model, param_grid_adaboost, cv=5, n_jobs=-1)
grid_search_adaboost.fit(X_train, y_train)

# Gunakan model AdaBoost dengan hyperparameter terbaik
best_adaboost_model = grid_search_adaboost.best_estimator_

# Evaluasi akurasi AdaBoost pada data pengujian
adaboost_accuracy = accuracy_score(y_test, best_adaboost_model.predict(X_test))
print("Akurasi AdaBoost:", adaboost_accuracy)

# Cetak hyperparameter terbaik untuk AdaBoost
best_adaboost_params = grid_search_adaboost.best_params_
print("Hyperparameter terbaik untuk AdaBoost:", best_adaboost_params)


c:\Users\shofiatul\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Akurasi AdaBoost: 0.7662337662337663
Hyperparameter terbaik untuk AdaBoost: {'learning_rate': 0.01, 'n_estimators': 200}


#### Tugas 3

Dengan menggunakan dataset diabetes, buatlah ensemble voting dengan algoritma
1. Logistic Regression
2. SVM kernel polynomial
3. Decission Tree

Anda boleh melakukan eksplorasi dengan melakukan tunning hyperparameter

In [46]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [47]:
# Load Data

dbt = pd.read_csv('diabetes.csv')

dbt.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [48]:
# Cek nama kolom
dbt.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [49]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose', 'BloodPlessure' ataupun 'Insulin'.
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

# Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
# Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


In [50]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns])

In [51]:
# Pisahkan fitur dan label
X = dbt.drop('Outcome', axis=1)
y = dbt['Outcome']

# Bagi dataset menjadi data pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


1. Training dengan Logistic Regression

In [52]:
# Inisialisasi model Logistic Regression
logistic_model = LogisticRegression()

# Buat ensemble voting classifier hanya dengan Logistic Regression
ensemble_classifier = VotingClassifier(
    estimators=[
        ('logistic', logistic_model)
    ],
    voting='hard'  # 'hard' voting digunakan untuk klasifikasi
)

# Latih ensemble classifier pada data pelatihan
ensemble_classifier.fit(X_train, y_train)

# Memprediksi label pada data pengujian
y_pred = ensemble_classifier.predict(X_test)

# Hitung akurasi ensemble classifier pada data pengujian
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi Ensemble Voting Classifier (Logistic Regression):", accuracy)


Akurasi Ensemble Voting Classifier (Logistic Regression): 0.7597402597402597


c:\Users\shofiatul\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


2. Training dengan SVM Kernel Polynomial

In [53]:
# Inisialisasi model SVM dengan kernel polynomial
svm_model = SVC(kernel='poly', degree=3)  # Kernel polynomial dengan derajat 3

# Latih model SVM pada data pelatihan
svm_model.fit(X_train, y_train)

# Memprediksi label pada data pengujian
y_pred = svm_model.predict(X_test)

# Hitung akurasi model SVM pada data pengujian
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi SVM dengan Kernel Polynomial:", accuracy)


Akurasi SVM dengan Kernel Polynomial: 0.7597402597402597


3. Training dengan Decision Tree

In [54]:
# Inisialisasi model Decision Tree
decision_tree_model = DecisionTreeClassifier(random_state=42)

# Latih model Decision Tree pada data pelatihan
decision_tree_model.fit(X_train, y_train)

# Memprediksi label pada data pengujian
y_pred = decision_tree_model.predict(X_test)

# Hitung akurasi model Decision Tree pada data pengujian
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi Decision Tree:", accuracy)


Akurasi Decision Tree: 0.7012987012987013
